In [1]:
## 11-14. 프로젝트: 한국어 데이터로 챗봇 만들기

# Step 1. 데이터 수집하기

# Step 2. 데이터 전처리하기
* 공백과  특수문자  처리,  토크나이징,  병렬데이터 구축의  과정이  적절히  진행되었다.

# Step 3. SubwordTextEncoder 사용하기

# Step 4. 모델 구성하기
* 구현한  트랜스포머  모델이  한국어  병렬  데이터 학습  시  안정적으로  수렴하였다.

# Step 5. 모델 평가하기
* 한국어  입력문장에  그럴듯한  한국어로 답변을 디코더  리턴하였다.